In [10]:
# 🎯 CIBLE : lien de la vidéo YouTube à booster en vues
video_link = "https://www.youtube.com/watch?v=FyznRjoS8pM"

# 📊 Script YouTube View Simulator – Selenium + Edge

## 🎯 Objectif
Simuler automatiquement des vues **réalistes** sur une vidéo YouTube en :
- Ouvrant Edge avec profil utilisateur temporaire
- Simulant des clics réels (boutons cookies, lecture)
- Attendant un temps de visionnage aléatoire
- Nettoyant chaque session pour relancer une vue propre

## ⚙️ Technologies
- `Selenium` (automatisation navigateur)
- `Edge WebDriver`
- `fake_useragent` (user-agent randomisé)
- `time`, `random`, `os`, `shutil`

## 🔁 Fonctionnement
- **watch_video_edge(url, watch_time=40)** :
  - Lance navigateur avec faux profil
  - Interagit avec les boutons "Lire la suite" et "Tout refuser"
  - Skippe la pub si détectée
  - Joue manuellement la vidéo si arrêtée
  - Attend un `watch_duration` ≥ 35 secondes

- **Boucle principale** :
  - `nb_views` fois → appelle la fonction
  - Affiche le numéro de vue
  - Pause entre 3 et 9 secondes entre chaque

## ⏱️ Temps estimé global
```python
estimated_total = nb_views * (40 + 45 + 9)
print(f"Temps max estimé : {estimated_total // 60}min {estimated_total % 60}s")

In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent
from tqdm import tqdm
import time
import random
import shutil
import os

# ✅ Vérifie s'il y a internet, sinon attend jusqu'à dispo

def wait_for_internet():
    while True:
        try:
            requests.get("https://www.google.com", timeout=3)
            print("✅ Connexion internet détectée")
            return
        except requests.ConnectionError:
            print("🔌 Pas de connexion... nouvelle tentative dans 5s")
            time.sleep(5)

# 🔁 LANCEMENT
wait_for_internet()  # ✅ Avant tout : vérifier la connexion
video_link = video_link

def watch_video_edge(video_url, watch_time=40, profile_index=0, force_duration=False):
    ua = UserAgent()
    profile_dir = f"user_profile_{profile_index}"
    if os.path.exists(profile_dir):
        shutil.rmtree(profile_dir)

    options = Options()
    options.use_chromium = True
    options.add_argument(f"user-agent={ua.random}")
    options.add_argument(f"--user-data-dir={os.path.abspath(profile_dir)}")
    options.add_argument("--disable-gpu")
    options.add_argument("--start-maximized")
    options.add_argument("--mute-audio") # Coupe le son
    options.add_argument("--headless")  # Masque le navigateur

    driver = webdriver.Edge(options=options)

    try:
        print(f"📺 Ouverture de {video_url}")
        driver.get(video_url)
        time.sleep(random.uniform(2.5, 5.0))

        # 🔘 Lire la suite
        try:
            time.sleep(random.uniform(1.0, 2.5))
            driver.execute_script("""
                let btns = [...document.querySelectorAll('button')];
                let lire = btns.find(b => b.innerText.trim().includes("Lire la suite"));
                if (lire) lire.click();
            """)
            print("📜 JS 'Lire la suite' cliqué")
            time.sleep(random.uniform(1.0, 2.5))
        except Exception as e:
            print("⛔ Erreur JS Lire la suite :", e)

        # 🔘 Tout refuser
        try:
            time.sleep(random.uniform(1.0, 2.5))
            driver.execute_script("""
                let btns = [...document.querySelectorAll('button')];
                let refuser = btns.find(b => b.innerText.trim().includes("Tout refuser"));
                if (refuser) refuser.click();
            """)
            print("🛡️ JS 'Tout refuser' cliqué")
            time.sleep(random.uniform(1.0, 2.5))
        except Exception as e:
            print("⛔ Erreur JS Tout refuser :", e)

        # ⏩ Skip pub
        try:
            time.sleep(random.uniform(4.0, 6.0))
            skip_btn = driver.find_element(By.CLASS_NAME, "ytp-ad-skip-button")
            skip_btn.click()
            print("⏩ Pub skippée")
            time.sleep(random.uniform(1.0, 2.5))
        except:
            print("⏳ Pas de pub à skipper")

        # ▶️ Lecture manuelle si arrêtée
        try:
            play_btn = driver.find_element(By.CSS_SELECTOR, "button.ytp-large-play-button")
            play_btn.click()
            print("▶️ Lecture manuelle déclenchée")
            time.sleep(random.uniform(1.5, 3.0))
        except:
            print("⏯️ Vidéo déjà en lecture ou bouton non détecté")

        # ⏱️ Visionnage
        if force_duration:
            watch_duration = watch_time + random.randint(1, 3)
        else:
            watch_duration = max(35, watch_time + random.randint(15, 45))

        print(f"⌛ Visionnage : {watch_duration}s")
        time.sleep(watch_duration)

        print("✅ Vue simulée")
    except Exception as e:
        print("❌ Erreur globale :", e)
    finally:
        driver.quit()
        if os.path.exists(profile_dir):
            shutil.rmtree(profile_dir)

# 🔁 LANCEMENT
video_link = video_link

# 🔁 Boucle jusqu'à saisie correcte d'une durée
while True:
    try:
        # 🧾 On demande à l'utilisateur la durée exacte de la vidéo :
        # - Si la vidéo dure moins de 30s → YouTube exige souvent de la **regarder jusqu’au bout**
        # - Si elle dure plus de 30s → une **vue est comptée après environ 30 secondes de visionnage réel**
        # Donc :
        # 👉 Si <35s = on entre la vraie durée
        # 👉 Sinon = on peut appuyer sur Entrée pour laisser 35s par défaut
        raw = input("⏱️ Durée de la vidéo (obligatoire si <35s, sinon entrer) : ").strip()

        # 🔢 Si une valeur est saisie → on la convertit
        # 🔁 Sinon on impose 35s par défaut (car YouTube valide une vue en général après ce seuil)
        video_duration = int(raw) if raw else 35

        # ❌ On empêche les entrées non valides (0, négatives, vides, caractères)
        if video_duration < 1:
            raise ValueError

        break  # ✅ Si tout est OK, on sort de la boucle
    except ValueError:
        # ⚠️ Feedback utilisateur si mauvaise saisie
        print("⛔ Entrez une durée valide supérieure à 0.")

# 🔢 Ensuite, on demande combien de vues on veut simuler
while True:
    try:
        nb_views = int(input("🔥 Combien de vues veux-tu simuler ? ").strip())
        if nb_views < 1:
            raise ValueError  # ❌ Interdit 0 ou négatif
        break
    except ValueError:
        print("⛔ Entrez un nombre valide strictement supérieur à 0.")

force_duration = video_duration < 30

# ⏳ Temps estimé global
estimated_total = nb_views * (video_duration + 45 + 9 if force_duration else 40 + 45 + 9)
h = estimated_total // 3600
m = (estimated_total % 3600) // 60
s = estimated_total % 60
print(f"\n⏳ Temps max estimé pour {nb_views} vues : {h}h {m}min {s}s\n")

for i in tqdm(range(nb_views), desc="📊 Progression", ncols=80):
    wait_for_internet()  # ✅ Avant chaque vue, on vérifie aussi
    print(f"\n🔥 Vue n°{i+1}")
    watch_video_edge(video_link, video_duration, profile_index=i, force_duration=force_duration)
    time.sleep(random.randint(3, 9))


⏳ Temps max estimé pour 1000 vues : 18h 53min 20s



📊 Progression:   0%|                                  | 0/1000 [00:00<?, ?it/s]


🔥 Vue n°1
📺 Ouverture de https://www.youtube.com/watch?v=FyznRjoS8pM
📜 JS 'Lire la suite' cliqué
🛡️ JS 'Tout refuser' cliqué
⏳ Pas de pub à skipper
▶️ Lecture manuelle déclenchée
⌛ Visionnage : 16s
✅ Vue simulée


📊 Progression:   0%|                       | 1/1000 [00:46<12:57:41, 46.71s/it]


🔥 Vue n°2
📺 Ouverture de https://www.youtube.com/watch?v=FyznRjoS8pM
📜 JS 'Lire la suite' cliqué
🛡️ JS 'Tout refuser' cliqué
⏳ Pas de pub à skipper
▶️ Lecture manuelle déclenchée
⌛ Visionnage : 17s
✅ Vue simulée


...


🔥 Vue n°283
📺 Ouverture de https://www.youtube.com/watch?v=FyznRjoS8pM
📜 JS 'Lire la suite' cliqué
🛡️ JS 'Tout refuser' cliqué
⏳ Pas de pub à skipper
▶️ Lecture manuelle déclenchée
⌛ Visionnage : 16s
✅ Vue simulée


📊 Progression:  28%|█████▋              | 283/1000 [3:54:41<9:57:31, 50.00s/it]


🔥 Vue n°284
📺 Ouverture de https://www.youtube.com/watch?v=FyznRjoS8pM
📜 JS 'Lire la suite' cliqué
🛡️ JS 'Tout refuser' cliqué
⏳ Pas de pub à skipper
▶️ Lecture manuelle déclenchée
⌛ Visionnage : 15s
✅ Vue simulée


📊 Progression:  28%|█████▋              | 284/1000 [3:55:24<9:32:20, 47.96s/it]


🔥 Vue n°285
📺 Ouverture de https://www.youtube.com/watch?v=FyznRjoS8pM
📜 JS 'Lire la suite' cliqué
🛡️ JS 'Tout refuser' cliqué
